# TP Coding Convolutional Neural Networks in tensorflow and keras - part 2

Author : Alasdair Newson

alasdair.newson@telecom-paristech.fr

In this session, we shall be looking at two subjects :

- A way to visualise what networks are learning : the Deep Dream algorithm
- Adversarial examples

For this, we shall use a famous pretrained network : VGG16. At the heart of these applications is the calculation of the gradient of a loss function with respect to the image itself (instead of respect to the weights). The loss function will be defined depending on the application at hand.

We can easily access certain well-known networks with the Keras programming framework. There are useful predefined function which allow us to load the weights, view the architecture etc. of the networks. We will specify these functions as necessary through the lab work. Unfortunately, the documentation for these functions is not very plentiful, but if you want to look at exactly what they do, you can look at the source code for help :

https://github.com/keras-team/keras-applications/blob/master/keras_applications/vgg16.py

https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py

However, we will indicate how these functions work as the need arises. You should fill in the code wherever you see :

#BEGIN STUDENT CODE
#END STUDENT CODE

First, let's load the necessary packages.

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img,img_to_array
from keras.applications import vgg16
from keras import backend as K
from keras.layers import Input
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

You'll notice that we have loaded the 'backend'. This allows us to access certain functions of the underlying backend layer without knowing the framework-specific syntax : Keras deals with everything. As we shall see, this will be useful later on.

Now, let's load one of the most famous networks, VGG16, and view it's architecture with the summary() function.

In [ ]:
# This removes all operations linked to training the model
K.set_learning_phase(0)
#load model. include_top=False means that we do not load the last fully connected layer(s) necessary for classification
model=vgg16.VGG16(weights='imagenet',include_top=False)

The 'summary' function gives a summary of the architecture so that we can reference its different layers

In [ ]:
model.summary()

You can extract one of the layers of the network with the following function

In [ ]:
#put the layers in an easy-to-reference dictionary
layer_dict = dict([ (layer.name,layer) for layer in model.layers])
#get the layer corresponding to the 5th block and second convolution
layer_out = layer_dict['block5_conv2'].output
layer_out.shape

You can also extract the weights of the convolutional filters in VGG16. You can do this with the following function :

- weights = layer.get_weights()

where 'layer' iterates throughout the 'model.layers' list. Note that this function returns the following :
- [weights, biases] if the layer is convolutional. weights has size [m,n,filter_depth,n_filters]
- an empty list otherwise

Write a function which retrieves the weights of the network (you will have to test if the weights variable is empty at each iteration).
    

In [ ]:

def retrieve_weights(model):
    """
    This function reads the weights of a pre
    
    Parameters
    ----------
    model : preloaded model
    
    Returns
    -------
    weight_list : list of numpy arrays
    list of the network filter weights
    """
    #BEGIN STUDENT CODE
    #END STUDENT CODE
    return weight_list

Now, write a code that visualises a single channel of a filter of your choice. Reminder, to view a grey-level image, you can use :

- plt.imshow(img,cmap="gray")
- plt.show()

In [ ]:
#BEGIN STUDENT CODE
#END STUDENT CODE

As you can probably see, this visualisation is not of much use : we cannot really tell what is going on in the network. For this, let's turn to another approach : Deep Dream !

## 2. Deep Dream

We now proceed to carry out the Deep Dream algorithm. The idea of the Deep Dream algorithm is to find an image which maximises the response of a network at a certain layer : $\textbf{this should help us understand what the network is learning}$. This can be done with an iterative algorithm, by simply carrying out gradient $\textbf{ascension}$. We start with an input image and iteratively add the gradient of the average response of the features which interest us. A pseudo-code for this would be :

- img = img_in
- for i=1:n_iters
    - img = img + grad_step $\nabla_{img} \mathcal{L}$,
    
where $\mathcal{L}$ is the average response which interest us (you need to define this).

Let's first define a function to preprocess the image. This is needed to put the image in the correct format for the VGG16 network. We also create a function to invert this process.

In [ ]:
def format_image(img_file):
    """
    This function reads and formats an image so that it can be fed to the VGG16 network
    
    Parameters
    ----------
    img_file : image file name
    
    Returns
    -------
    img_out_vgg : the correctly formatted image for VGG16
    img : the image as read by the load_img function of keras.preprocessing.image
    """
    # read image
    img = load_img(img_file)
    # convert image to an array
    img_out = img_to_array(img)
    # preprocess the image to put in the correct format for use with the VGG16 network trained on imagenet
    img_out_vgg = vgg16.preprocess_input(img_out)
    # add a dimension at the beginning, coresponding to the batch dimension
    img_out_vgg = np.expand_dims(img_out_vgg, axis=0)
    return img_out_vgg, img

def unformat_image(img_in):
    """
    This function inverts the preprocessing applied to images for use in the VGG16 network
    
    Parameters
    ----------
    img_file : formatted image of shape (batch_size,m,n,3)
    
    Returns
    -------
    img_out : a m-by-n-by-3 array, representing an image that can be written to an image file
    """
    #get rid of batch dimension
    img_out=np.squeeze(img_in)
    #remove offsets added by the VGG16 preprocessing
    img_out[:, :, 0] += 103.939
    img_out[:, :, 1] += 116.779
    img_out[:, :, 2] += 123.68
    # invert the order of the colours : BGR -> RGB
    img_out = img_out[:, :, ::-1]
    #clamp image to the range [0,255] and cast to uint8
    img_out = np.clip(img_out, 0, 255).astype('uint8')
    return img_out

Now, we load the image. At the same time, we create a backend Tensor which has the correct format for the network

In [ ]:
#load image
img_in,img_visu = format_image('got.jpg')
#create a Tensor in the correct format for the network using the backend framework
img_backend = model.input
#show the input image
plt.imshow(img_visu)
plt.show()

Now, we define the loss that we wish to maximise. This can be anything you wish, but the a common loss is simply the average response of a certain channel of a certain layer. Since these responses are all positive, due to the non-linearities used, we can safely take the average as a loss function to maximise.

Define the loss as the average response of the 15th channel of the 5th layer, second convolution. Then define the gradients of loss with respect to the image. You can retrieve the responses inside the network via the 'layer_dict' variable defined above.

Note that all these operations should be done using the backend functionalities, since we are working on symbolic Tensors which have not yet been given numeric data. For example, use the following functions :
- K.mean()
- K.gradients(loss, image_variable)[0]  #we take the [0]th element here because the gradients function necessarily returns a list, even when the length of this list is 1

In [ ]:
#BEGIN STUDENT CODE
#define loss : average response of the 15th channel of the 5th layer, second convolution
loss = ...
#define gradient of the loss with respect to 
grads = ...
#END STUDENT CODE

Here, we use a little trick. Indeed, it may be the case that the gradients are far too small or far too large for updating, meaning that the updates do nothing or destroy the image. To avoid this, we normalise the gradients. We normalise the gradient in the following manner :

- grad = grad/(max(mean(abs(grad)))

In [ ]:
# normalize the gradient
grads_normalised = grads/ (K.maximum(K.mean(K.abs(grads)), K.epsilon()))

Now, we need to define a function to retrieve the loss and gradients so as to maximise our loss function. This can done easily in Keras with the following syntax

In [ ]:
# this defines a function to retrieve the loss and normalised gradients of the loss with respect to the image
get_loss_and_grads = K.function([img_backend],[loss,grads_normalised])

We are now ready to carry out the Deep Dream algorithm using gradient ascent, yipee ! Iterate 'n_iterations' times, each time adding an epsilon of the gradient. 

In [ ]:
# first, reload image to make sure that we are not starting from a previous initialisation
img_in,_ = format_image('got.jpg')
# Gradient ascent step size
step = 0.1
# Number of gradient ascent steps
n_iterations = 30  
for ii in range(0,n_iterations):
    #BEGIN STUDENT CODE
    #calculate the gradient
    ...
    #carry out one step of gradient ascent
    ...
    #END STUDENT CODE
    #save result every 5 iterations
    if (ii%5==0):
        print(".", end='')
        img_out = unformat_image(np.copy(img_in))
        plt.imsave('img_out_'+str(ii).zfill(3)+'.png',img_out)
        plt.imshow(img_out)

You can try different convolutional layers and see what the results are !

## 3. Adversarial examples

In this part of the lab work, we will explore the interesting case of adversarial examples. Adversarial examples are images which have been perturbed in a manner which makes the network misclassify the image.

There are many ways to do this, however we can use a similar approach to the one used above, that is to say, we will use a gradient maximisation approach. This consists in iteratively adding the gradient of the loss with respect to the image, to the current image, in order to get a misclassified image.

For this application, we need access to the last, classification, layer of the VGG16 network.

In [ ]:
# load model, including last (classification layer)
model=vgg16.VGG16(weights='imagenet',include_top=True)
#create layer dictionary
layer_dict = dict([ (layer.name,layer) for layer in model.layers])
#create backend Tensor
img_backend = model.input
# display architecture
model.summary()

We are going to take an image of a cat and try to misclassify it. First, read and format the image.

$\textbf{NOTE !!!}$

In the previous code, we were not interested in the fully connected layers, so the input image could be of any size (there were only convolutional and maxpool layers). Now, since we are using the fully connected layers, we need to make sure the input is of the correct size for the images in imagenet : 224x224. For this, we redefine the format_image function to force the image to a certain size

In [ ]:
def format_image_classif(img_file,img_width=224,img_height=224):
    """
    This function reads and formats an image so that it can be fed to the VGG16 network.
    In this case, we wish to force the image size to a certain shape, since we want to use the image for
    classification
    
    Parameters
    ----------
    img_file : image file name
    img_width : the target image width
    img_height : he target image height
    
    Returns
    -------
    img_out_vgg : the correctly formatted image for VGG16
    img : the image as read by the load_img function of keras.preprocessing.image
    """
    # read image. Force the image size to a certain shape (uses a resize of the pillow package)
    img = load_img(img_file,target_size=(img_height,img_width))
    # convert image to an array
    img_out = img_to_array(img)
    # preprocess the image to put in the correct format for use with the VGG16 network trained on imagenet
    img_out_vgg = vgg16.preprocess_input(img_out)
    # add a dimension at the beginning, coresponding to the batch dimension
    img_out_vgg = np.expand_dims(img_out_vgg, axis=0)
    return img_out_vgg, img

In [ ]:
img_in,img_visu = format_image_classif('cat_small.png')
plt.imshow(img_visu)

Now, we need to retrieve the last (prediction) layer of the VGG network. Do this via the variable layer_dict created above, and calling the correct layer (similarly to when we called the 'block5_conv2' layer above)

In [ ]:
#BEGIN STUDENT CODE
#get last layer (prediction layer)
last_layer = layer_dict['predictions'].output
#create a function to get last layer of the VGG16 network (using the backend function)
get_prediction_layer = K.function([img_backend],[last_layer])
#END STUDENT CODE

We are going to try to force the image to recognise a 'reflex_camera'. This is number 759 of the imagenet classes. You can use any one you like in fact (apart from ones linked to cats, obviously). To see the list of classes go to :

https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

Now, find the initial class of the cat image. Also display the top 5 classification results. For this, you can use the following functions :

- model.predict(img) : classification prediction of the img variable
- vgg16.decode_predictions(y, top=5)[0] : converts the numerical probabilities in the y variable to human readable classes. Here,

In [ ]:
target_class = 759
# BEGIN STUDENT CODE
# carry out the network predictions on the example image
...
#define the true class as the initial most likely class
...
# print the top 5 predicted classes, with the prediction probability
...
# END STUDENT CODE

Now, redefine the loss to be the 'target_class' element of the last layer of the network. This is the element we shall try to maximise, in order to fool the network. Redefine also the (normalised) gradient as above, and create a function to extract the loss and the gradients, given an input image, as above.

In [ ]:
#BEGIN STUDENT CODE
#END STUDENT CODE

We are now ready to perturb the image such that we misclassify it. Modify the following code to carry out the gradient ascent.

In [ ]:
#reload image, to make sure we are not starting from previous point
img_in,_ = format_image_classif('cat_small.png')
#parameters
step = 0.5  # Gradient ascent step size
n_iterations = 100  # Number of gradient ascent steps
for ii in range(0,n_iterations):
    #BEGIN STUDENT CODE
    #END STUDENT CODE
    if (ii%5==0):
        img_show = unformat_image(np.copy(img_in))
        plt.imsave('img_out_'+str(ii).zfill(3)+'.png',img_show)
        #predict current model to see evolution of top classification
        y_predicted = model.predict(img_in)
        print(vgg16.decode_predictions(y_predicted, top=1)[0])
print('End of optimisation')
# finally, display misclassified image
img_show = unformat_image(np.copy(img_in))
plt.imshow(img_show)
#show the final top 5 classes
y_predicted = model.predict(img_in)
print(vgg16.decode_predictions(y_predicted, top=5)[0])

As you should probably see, the image is changed such that it is no longer is correctly classified. It should be classified as a 'reflex camera', or whatever you chose, with high probability. This is a problem, since a human is still able to see a cat ! Furthermore, the top 5 classifications have nothing to do with cats !! Even bigger problem !!!